In [42]:
!pip install z3-solver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
from itertools import combinations
from z3 import *
import utils
import math

# Define problem parameters

In [45]:
v = 9
items = [5,6,5,3,8,1,1,4]
numberOfItems = len(items)
couriers = [15,7,5]
numberOfCouriers = len(couriers)
distance = [[0 ,1 ,3 ,4 ,2 ,5 ,6 ,6 ,4 ,6],
            [1 ,0 ,4 ,2 ,5 ,6 ,3 ,3 ,1 ,5],
            [3 ,4 ,0 ,9 ,8 ,9 ,5 ,7 ,1 ,8],
            [4 ,2 ,9 ,0 ,1 ,2 ,3 ,2 ,2 ,5],
            [2 ,5 ,8 ,1 ,0 ,4 ,3 ,7 ,5 ,3],
            [5 ,6 ,9 ,2 ,4 ,0 ,8 ,1 ,8 ,7],
            [6 ,3 ,5 ,3 ,3 ,8 ,0 ,4 ,8 ,9],
            [6 ,3 ,7 ,2 ,7 ,1 ,4 ,0 ,4 ,1],
            [4 ,1 ,1 ,2 ,5 ,8 ,8 ,4 ,0 ,8],
            [6 ,5 ,8 ,5 ,3 ,7 ,9 ,1 ,8 ,0]]
# represents whether an item is assigned to a courier
path = [[Bool(f"p_{i}_{j}") for j in range(numberOfCouriers)] for i in range(numberOfItems)]
length = [Int(f"l_{i}") for i in range(numberOfCouriers)]

s = Solver()

## Each item is delivered by exactly one courier


In [46]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

In [47]:
for item in range(numberOfItems):
      s.add(exactly_one([path[item][courier] for courier in range(numberOfCouriers)]))

## Each courier delivers no more items than their capacity.


In [48]:
def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

In [49]:
for item in range(numberOfItems):
   for courier in range(numberOfCouriers):
      s.add(at_most_k_np([path[item][courier]],couriers[courier]))

## Calculates the length of the path for each courier and sets the corresponding length variable to that value

In [51]:
if s.check() == sat:
    m = s.model()
    print(couriers)
    for c in range(numberOfCouriers):
      for i in range(numberOfItems):
        if m.evaluate(path[i][c]):
          print(f"item {i} is delivered by {c}")
      
else:
    print("unsat")

[15, 7, 5]
item 0 is delivered by 0
item 1 is delivered by 0
item 2 is delivered by 0
item 3 is delivered by 0
item 4 is delivered by 0
item 5 is delivered by 0
item 7 is delivered by 0
item 6 is delivered by 1
